In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(1)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 4

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
def DRM(x):
    u_hat = model(x)
    ux = torch.zeros(x.size()[0], dimension).cuda()
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum(0.5 * uxsq + 0.5*pi**2*u_hat**2 - f_temp*u_hat)/x.size()[0]
    
    Nb = 100
    xa1 = torch.rand(Nb, dimension).cuda() 
    xa1[:, 0] = torch.zeros(Nb)
    xa2 = torch.rand(Nb, dimension).cuda() 
    xa2[:, 0] = torch.ones(Nb)
    xb1 = torch.rand(Nb, dimension).cuda()
    xb1[:, 1] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension).cuda()
    xb2[:, 1] = torch.ones(Nb)
    xc1 = torch.rand(Nb, dimension).cuda()
    xc1[:, 2] = torch.zeros(Nb)
    xc2 = torch.rand(Nb, dimension).cuda()
    xc2[:, 2] = torch.ones(Nb)
    xd1 = torch.rand(Nb, dimension).cuda()
    xd1[:, 3] = torch.zeros(Nb)
    xd2 = torch.rand(Nb, dimension).cuda()
    xd2[:, 3] = torch.ones(Nb)
    sa1 = (torch.sum((model(xa1)-u_ex(xa1))**2)/xa1.size()[0]).cuda()
    sa2 = (torch.sum((model(xa2)-u_ex(xa2))**2)/xa1.size()[0]).cuda() 
    sb1 = (torch.sum((model(xb1)-u_ex(xb1))**2)/xa1.size()[0]).cuda() 
    sb2 = (torch.sum((model(xb2)-u_ex(xb2))**2)/xa1.size()[0]).cuda()
    sc1 = (torch.sum((model(xc1)-u_ex(xc1))**2)/xa1.size()[0]).cuda() 
    sc2 = (torch.sum((model(xc2)-u_ex(xc2))**2)/xa1.size()[0]).cuda()
    sd1 = (torch.sum((model(xd1)-u_ex(xd1))**2)/xa1.size()[0]).cuda() 
    sd2 = (torch.sum((model(xd2)-u_ex(xd2))**2)/xa1.size()[0]).cuda()
    part_2 = sa1 + sa2 + sb1 + sb2 + sc1 + sc2 + sd1 + sd2
    
    lambda1 = 100.0
    return part_1 + lambda1 * part_2 / 8

In [10]:
Data_size = 2000
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x.cuda()

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 8)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:1" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=4, out_features=8, bias=True)
  (linear1): Linear(in_features=8, out_features=8, bias=True)
  (linear2): Linear(in_features=8, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=8, bias=True)
  (linear4): Linear(in_features=8, out_features=8, bias=True)
  (linear5): Linear(in_features=8, out_features=8, bias=True)
  (linear6): Linear(in_features=8, out_features=8, bias=True)
  (linear_out): Linear(in_features=8, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [16]:
traintime = 20000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_4D_Dirichlet.npy", error_save)
np.save("DRM_relative_error_4D_Dirichlet.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(886.1710, device='cuda:0')
current relative error is:  tensor(1.8904, device='cuda:0')
current epoch is:  50
current loss is:  tensor(258.6480, device='cuda:0')
current relative error is:  tensor(0.9860, device='cuda:0')
current epoch is:  100
current loss is:  tensor(184.4542, device='cuda:0')
current relative error is:  tensor(0.8421, device='cuda:0')
current epoch is:  150
current loss is:  tensor(75.0723, device='cuda:0')
current relative error is:  tensor(0.5627, device='cuda:0')
current epoch is:  200
current loss is:  tensor(-1.6778, device='cuda:0')
current relative error is:  tensor(0.2190, device='cuda:0')
current epoch is:  250
current loss is:  tensor(-4.9213, device='cuda:0')
current relative error is:  tensor(0.1864, device='cuda:0')
current epoch is:  300
current loss is:  tensor(-5.7401, device='cuda:0')
current relative error is:  tensor(0.1730, device='cuda:0')
current epoch is:  350
current loss is:  tensor(-8.4429, devic

current epoch is:  3050
current loss is:  tensor(-15.3492, device='cuda:0')
current relative error is:  tensor(0.1212, device='cuda:0')
current epoch is:  3100
current loss is:  tensor(-15.6080, device='cuda:0')
current relative error is:  tensor(0.1217, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(-16.6085, device='cuda:0')
current relative error is:  tensor(0.1193, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(-16.4157, device='cuda:0')
current relative error is:  tensor(0.1171, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(-16.7218, device='cuda:0')
current relative error is:  tensor(0.1155, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(-15.0533, device='cuda:0')
current relative error is:  tensor(0.1175, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(-17.3298, device='cuda:0')
current relative error is:  tensor(0.1164, device='cuda:0')
current epoch is:  3400
current loss is:  tensor

current epoch is:  6100
current loss is:  tensor(-18.7667, device='cuda:0')
current relative error is:  tensor(0.0394, device='cuda:0')
current epoch is:  6150
current loss is:  tensor(-20.2089, device='cuda:0')
current relative error is:  tensor(0.0469, device='cuda:0')
current epoch is:  6200
current loss is:  tensor(-20.5816, device='cuda:0')
current relative error is:  tensor(0.0331, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(-18.4332, device='cuda:0')
current relative error is:  tensor(0.0309, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(-18.7400, device='cuda:0')
current relative error is:  tensor(0.0284, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(-20.4946, device='cuda:0')
current relative error is:  tensor(0.0286, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(-19.4796, device='cuda:0')
current relative error is:  tensor(0.0283, device='cuda:0')
current epoch is:  6450
current loss is:  tensor

current epoch is:  9150
current loss is:  tensor(-20.1742, device='cuda:0')
current relative error is:  tensor(0.0200, device='cuda:0')
current epoch is:  9200
current loss is:  tensor(-20.1572, device='cuda:0')
current relative error is:  tensor(0.0151, device='cuda:0')
current epoch is:  9250
current loss is:  tensor(-20.2444, device='cuda:0')
current relative error is:  tensor(0.0172, device='cuda:0')
current epoch is:  9300
current loss is:  tensor(-19.4526, device='cuda:0')
current relative error is:  tensor(0.0152, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(-19.6909, device='cuda:0')
current relative error is:  tensor(0.0225, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(-20.5753, device='cuda:0')
current relative error is:  tensor(0.0152, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(-20.9184, device='cuda:0')
current relative error is:  tensor(0.0155, device='cuda:0')
current epoch is:  9500
current loss is:  tensor

current epoch is:  12150
current loss is:  tensor(-20.0144, device='cuda:0')
current relative error is:  tensor(0.0130, device='cuda:0')
current epoch is:  12200
current loss is:  tensor(-19.8465, device='cuda:0')
current relative error is:  tensor(0.0117, device='cuda:0')
current epoch is:  12250
current loss is:  tensor(-19.7381, device='cuda:0')
current relative error is:  tensor(0.0130, device='cuda:0')
current epoch is:  12300
current loss is:  tensor(-20.0729, device='cuda:0')
current relative error is:  tensor(0.0122, device='cuda:0')
current epoch is:  12350
current loss is:  tensor(-18.1564, device='cuda:0')
current relative error is:  tensor(0.0126, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(-20.1628, device='cuda:0')
current relative error is:  tensor(0.0155, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(-20.2402, device='cuda:0')
current relative error is:  tensor(0.0166, device='cuda:0')
current epoch is:  12500
current loss is:

current epoch is:  15150
current loss is:  tensor(-18.2172, device='cuda:0')
current relative error is:  tensor(0.0114, device='cuda:0')
current epoch is:  15200
current loss is:  tensor(-19.8814, device='cuda:0')
current relative error is:  tensor(0.0106, device='cuda:0')
current epoch is:  15250
current loss is:  tensor(-20.8488, device='cuda:0')
current relative error is:  tensor(0.0102, device='cuda:0')
current epoch is:  15300
current loss is:  tensor(-20.2836, device='cuda:0')
current relative error is:  tensor(0.0109, device='cuda:0')
current epoch is:  15350
current loss is:  tensor(-21.4120, device='cuda:0')
current relative error is:  tensor(0.0155, device='cuda:0')
current epoch is:  15400
current loss is:  tensor(-20.8137, device='cuda:0')
current relative error is:  tensor(0.0109, device='cuda:0')
current epoch is:  15450
current loss is:  tensor(-19.8297, device='cuda:0')
current relative error is:  tensor(0.0104, device='cuda:0')
current epoch is:  15500
current loss is:

current epoch is:  18150
current loss is:  tensor(-18.7661, device='cuda:0')
current relative error is:  tensor(0.0098, device='cuda:0')
current epoch is:  18200
current loss is:  tensor(-19.1364, device='cuda:0')
current relative error is:  tensor(0.0101, device='cuda:0')
current epoch is:  18250
current loss is:  tensor(-19.1353, device='cuda:0')
current relative error is:  tensor(0.0116, device='cuda:0')
current epoch is:  18300
current loss is:  tensor(-19.0080, device='cuda:0')
current relative error is:  tensor(0.0092, device='cuda:0')
current epoch is:  18350
current loss is:  tensor(-20.6016, device='cuda:0')
current relative error is:  tensor(0.0096, device='cuda:0')
current epoch is:  18400
current loss is:  tensor(-18.6233, device='cuda:0')
current relative error is:  tensor(0.0121, device='cuda:0')
current epoch is:  18450
current loss is:  tensor(-20.5047, device='cuda:0')
current relative error is:  tensor(0.0106, device='cuda:0')
current epoch is:  18500
current loss is: